<h2>Simple LSTM RNN project</h2>
<p>The below code is a simple LSTM RNN project</p>

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [3]:
# Step 1: Data Collection
# Fetch historical stock price data for Apple Inc.
ticker = 'AAPL'
start_date = '2010-01-01'
end_date = '2021-01-01'
data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [4]:
# Step 2: Data Preprocessing
data = data[['Close']]
scaler = MinMaxScaler()
data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

look_back = 30
X_train, y_train = create_dataset(train_data.values, look_back)
X_test, y_test = create_dataset(test_data.values, look_back)

<ipython-input-4-f3851767cccc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))


In [5]:
# Step 3: Build the LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [6]:
# Step 4: Model Training
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
69/69 [==============================] - 6s 33ms/step - loss: 0.0024
Epoch 2/50
69/69 [==============================] - 4s 52ms/step - loss: 6.1275e-05
Epoch 3/50
69/69 [==============================] - 2s 33ms/step - loss: 5.8775e-05
Epoch 4/50
69/69 [==============================] - 2s 31ms/step - loss: 5.5522e-05
Epoch 5/50
69/69 [==============================] - 2s 34ms/step - loss: 5.7323e-05
Epoch 6/50
69/69 [==============================] - 2s 34ms/step - loss: 5.4632e-05
Epoch 7/50
69/69 [==============================] - 3s 49ms/step - loss: 5.5116e-05
Epoch 8/50
69/69 [==============================] - 3s 36ms/step - loss: 5.0958e-05
Epoch 9/50
69/69 [==============================] - 2s 34ms/step - loss: 5.2757e-05
Epoch 10/50
69/69 [==============================] - 2s 34ms/step - loss: 4.9371e-05
Epoch 11/50
69/69 [==============================] - 2s 33ms/step - loss: 4.7459e-05
Epoch 12/50
69/69 [==============================] - 4s 62ms/step - loss: 5.06

In [7]:
# Step 5: Model Evaluation
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform the predictions to get original scale
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calculate RMSE (Root Mean Squared Error)
from sklearn.metrics import mean_squared_error
train_rmse = np.sqrt(mean_squared_error(train_data[look_back:], train_predict))
test_rmse = np.sqrt(mean_squared_error(test_data[look_back:], test_predict))

print(f'Training RMSE: {train_rmse}')
print(f'Testing RMSE: {test_rmse}')

17/17 [==============================] - 0s 14ms/step
Training RMSE: 27.230700190456073
Testing RMSE: 77.06402793435117
